# SpatialLM Inference Notebook
This notebook breaks down the inference.py workflow into interactive cells.

In [19]:
# Cell 1: Imports and Constants
import os
import sys
import glob
import json
from pathlib import Path

import torch
import numpy as np
from tqdm import tqdm
from threading import Thread
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import TextIteratorStreamer, set_seed

# Add project root to path
PROJECT_ROOT = Path("/home/jixu233b/Projects/VLM_3D/SpatialLM")
sys.path.insert(0, str(PROJECT_ROOT))

from spatiallm import Layout
from spatiallm.pcd import load_o3d_pcd, get_points_and_colors, cleanup_pcd, Compose

DETECT_TYPE_PROMPT = {
    "all": "Detect walls, doors, windows, boxes.",
    "arch": "Detect walls, doors, windows.",
    "object": "Detect boxes.",
}

print("✓ Imports loaded")

✓ Imports loaded


In [20]:
# Cell 2: Define Helper Functions

def preprocess_point_cloud(points, colors, grid_size, num_bins):
    """Preprocess point cloud into tensor features."""
    transform = Compose([
        dict(type="PositiveShift"),
        dict(type="NormalizeColor"),
        dict(
            type="GridSample",
            grid_size=grid_size,
            hash_type="fnv",
            mode="test",
            keys=("coord", "color"),
            return_grid_coord=True,
            max_grid_coord=num_bins,
        ),
    ])
    point_cloud = transform({
        "name": "pcd",
        "coord": points.copy(),
        "color": colors.copy(),
    })
    coord = point_cloud["grid_coord"]
    xyz = point_cloud["coord"]
    rgb = point_cloud["color"]
    point_cloud = np.concatenate([coord, xyz, rgb], axis=1)
    return torch.as_tensor(np.stack([point_cloud], axis=0))


def generate_layout(model, point_cloud, tokenizer, code_template_file, 
                    top_k=10, top_p=0.95, temperature=0.6, num_beams=1,
                    seed=-1, max_new_tokens=4096, detect_type="all", categories=[]):
    """Generate layout from point cloud using the model."""
    if seed >= 0:
        set_seed(seed)
    
    with open(code_template_file, "r") as f:
        code_template = f.read()
    
    task_prompt = DETECT_TYPE_PROMPT[detect_type]
    if detect_type != "arch" and categories:
        task_prompt = task_prompt.replace("boxes", ", ".join(categories))
    print("Task prompt:", task_prompt)
    
    prompt = f"<|point_start|><|point_pad|><|point_end|>{task_prompt} The reference code is as followed: {code_template}"
    
    if model.config.model_type == "spatiallm_qwen":
        conversation = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt},
        ]
    else:
        conversation = [{"role": "user", "content": prompt}]
    
    input_ids = tokenizer.apply_chat_template(conversation, add_generation_prompt=True, return_tensors="pt")
    input_ids = input_ids.to(model.device)
    
    streamer = TextIteratorStreamer(tokenizer, timeout=20.0, skip_prompt=True, skip_special_tokens=True)
    
    generate_kwargs = dict(
        {"input_ids": input_ids, "point_clouds": point_cloud},
        streamer=streamer,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        use_cache=True,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
    )
    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()
    
    print("Generating layout...\\n")
    generate_texts = []
    for text in streamer:
        generate_texts.append(text)
        print(text, end="", flush=True)
    print("\\nDone!")
    
    layout_str = "".join(generate_texts)
    layout = Layout(layout_str)
    layout.undiscretize_and_unnormalize(num_bins=model.config.point_config["num_bins"])
    return layout

print("✓ Helper functions defined")

✓ Helper functions defined


In [21]:
# Cell 3: Set Parameters
DATA_ROOT = Path("/data/horse/ws/jixu233b-metadata_ws/datasets/arkitscenes-spatiallm/")

# Input/Output paths
point_cloud_path = DATA_ROOT / "pcd/40753679.ply"
output_path = PROJECT_ROOT / "outputs/scene40753679.txt"

# Model configuration
model_path = "manycore-research/SpatialLM1.1-Qwen-0.5B"

# Inference parameters
detect_type = "all"
categories = []
code_template_file = str(PROJECT_ROOT / "code_template.txt")

# Generation parameters
top_k = 10
top_p = 0.95
temperature = 0.6
num_beams = 1
seed = -1
max_new_tokens = 4096
inference_dtype = "bfloat16"
no_cleanup = False

# Optional: JSON file to filter specific samples
json_file = None

print("✓ Parameters configured")

✓ Parameters configured


In [22]:
# Cell 4: Load Model and Tokenizer
print(f"Loading model from {model_path}...")
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=getattr(torch, inference_dtype))
model.to("cuda")
model.set_point_backbone_dtype(torch.float32)
model.eval()

num_bins = model.config.point_config["num_bins"]

print(f"✓ Model loaded")
print(f"  - Type: {model.config.model_type}")
print(f"  - Bins: {num_bins}")
print(f"  - Device: {model.device}")

Loading model from manycore-research/SpatialLM1.1-Qwen-0.5B...


RuntimeError: No CUDA GPUs are available

In [ ]:
# Cell 5: Discover Point Cloud Files
if os.path.isfile(point_cloud_path):
    point_cloud_files = [point_cloud_path]
else:
    if json_file is not None:
        print(f"Loading samples from JSON: {json_file}")
        with open(json_file, "r") as f:
            json_data = json.load(f)
        
        point_cloud_files = []
        for item in json_data:
            pcd_files = []
            if isinstance(item, str):
                pcd_files = [item]
            elif isinstance(item, dict):
                value = (item.get("point_clouds") or item.get("point_cloud") or 
                        item.get("file_name") or item.get("pcd_file") or 
                        item.get("scene_id") or item.get("id"))
                pcd_files = value if isinstance(value, list) else [value] if value else []
            
            for pcd_file in pcd_files:
                pcd_file = pcd_file.replace("pcd/", "")
                if not pcd_file.endswith(".ply"):
                    pcd_file = f"{pcd_file}.ply"
                
                full_path = os.path.join(point_cloud_path, pcd_file)
                if os.path.exists(full_path):
                    point_cloud_files.append(full_path)
                else:
                    print(f"Warning: Not found: {full_path}")
        print(f"Found {len(point_cloud_files)} samples from JSON")
    else:
        point_cloud_files = glob.glob(os.path.join(point_cloud_path, "*.ply"))

print(f"✓ Found {len(point_cloud_files)} file(s)")
for i, f in enumerate(point_cloud_files[:5]):
    print(f"  [{i+1}] {os.path.basename(f)}")
if len(point_cloud_files) > 5:
    print(f"  ... +{len(point_cloud_files)-5} more")

In [ ]:
# Cell 6: Load and Preprocess Point Cloud
current_file_idx = 0
point_cloud_file = point_cloud_files[current_file_idx]
print(f"Processing: {os.path.basename(point_cloud_file)}")

point_cloud = load_o3d_pcd(point_cloud_file)
grid_size = Layout.get_grid_size(num_bins)

if not no_cleanup:
    point_cloud = cleanup_pcd(point_cloud, voxel_size=grid_size)

points, colors = get_points_and_colors(point_cloud)
min_extent = np.min(points, axis=0)

print(f"✓ Point cloud loaded")
print(f"  - Points: {len(points)}")
print(f"  - Range: {np.min(points, axis=0)} to {np.max(points, axis=0)}")
print(f"  - Grid: {grid_size}")

input_pcd = preprocess_point_cloud(points, colors, grid_size, num_bins)
print(f"✓ Preprocessed: {input_pcd.shape}")

In [ ]:
# Cell 7: Generate Layout
layout = generate_layout(
    model, input_pcd, tokenizer, code_template_file,
    top_k=top_k, top_p=top_p, temperature=temperature,
    num_beams=num_beams, seed=seed, detect_type=detect_type,
    categories=categories
)

layout.translate(min_extent)
print(f"\\n✓ Layout generated")

In [ ]:
# Cell 8: Save Output
pred_language_string = layout.to_language_string()

if os.path.splitext(output_path)[-1]:
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, "w") as f:
        f.write(pred_language_string)
    print(f"✓ Saved to: {output_path}")
else:
    output_filename = os.path.basename(point_cloud_file).replace(".ply", ".txt")
    os.makedirs(output_path, exist_ok=True)
    output_file = os.path.join(output_path, output_filename)
    with open(output_file, "w") as f:
        f.write(pred_language_string)
    print(f"✓ Saved to: {output_file}")

print("\\n--- Preview (500 chars) ---")
print(pred_language_string[:500])
if len(pred_language_string) > 500:
    print(f"... +{len(pred_language_string)-500} chars")

## Optional: Batch Processing
Uncomment the cell below to process all files in batch mode.

In [ ]:
# Cell 9: Batch Process All Files (Optional)
# Uncomment to run batch processing

# for point_cloud_file in tqdm(point_cloud_files):
#     point_cloud = load_o3d_pcd(point_cloud_file)
#     grid_size = Layout.get_grid_size(num_bins)
#     if not no_cleanup:
#         point_cloud = cleanup_pcd(point_cloud, voxel_size=grid_size)
#     points, colors = get_points_and_colors(point_cloud)
#     min_extent = np.min(points, axis=0)
#     input_pcd = preprocess_point_cloud(points, colors, grid_size, num_bins)
#     layout = generate_layout(model, input_pcd, tokenizer, code_template_file,
#                              top_k=top_k, top_p=top_p, temperature=temperature,
#                              num_beams=num_beams, seed=seed, detect_type=detect_type,
#                              categories=categories)
#     layout.translate(min_extent)
#     pred_language_string = layout.to_language_string()
#     if os.path.splitext(output_path)[-1]:
#         with open(output_path, "w") as f:
#             f.write(pred_language_string)
#     else:
#         output_filename = os.path.basename(point_cloud_file).replace(".ply", ".txt")
#         os.makedirs(output_path, exist_ok=True)
#         with open(os.path.join(output_path, output_filename), "w") as f:
#             f.write(pred_language_string)
# print(f"✓ Batch complete: {len(point_cloud_files)} files")